In [171]:
import cv2

# read all the file names in the folder named CNV and store them in a list
import glob
cnv = glob.glob("CNV/*.jpeg")
dme = glob.glob("DME/*.jpeg")
drusen = glob.glob("DRUSEN/*.jpeg")
normal = glob.glob("NORMAL/*.jpeg")

# get only top 1000 images from each category
cnv = cnv[:1000]
dme = dme[:1000]
drusen = drusen[:1000]
normal = normal[:1000]


# read all the images and store them in a list
cnv_images = [cv2.imread(img) for img in cnv]
# print("Read all the images from CNV folder")
dme_images = [cv2.imread(img) for img in dme]
# print("Read all the images from DME folder")
drusen_images = [cv2.imread(img) for img in drusen]
# print("Read all the images from DRUSEN folder")
normal_images = [cv2.imread(img) for img in normal]
# print("Read all the images from NORMAL folder")


In [3]:

import cv2
def preProcess(img):
    # use matplotlib to plot the first image from cnv_images
    from PIL import Image
    import matplotlib.pyplot as plt
    # plt.imshow(img)
    # plt.show()


    # applymedian filter to the image
    import numpy as np
    median = cv2.medianBlur(img, 5)
    # plt.imshow(median)
    # plt.show()


    # ehnance the image using another library
    from PIL import ImageEnhance
    enhancer = ImageEnhance.Contrast(Image.fromarray(median))
    enhanced_im = enhancer.enhance(2.0)
    # plt.imshow(enhanced_im)
    # plt.show()

    # apply thresholding to the enhanced image
    
    ret,thresh1 = cv2.threshold(np.array(enhanced_im), 50, 255, cv2.THRESH_BINARY)
    # plt.imshow(thresh1)
    # plt.show()

    # set the pixels of cnv_images[0] to 0 where thresh1 is 0
    img[thresh1==0] = 0
    # plt.imshow(img)
    # plt.show()

    # crop img 100 from top
    img = img[50:,:]
    # plt.imshow(img)
    # plt.show()
    # crop img 100 from bottom
    img = img[:-50,:]
    # plt.imshow(img)
    # plt.show()

    # resize the image to 256x256
    img = cv2.resize(img, (256,256))
    # plt.imshow(img)
    # plt.show()

    # apply cannny edge detection
    edges = cv2.Canny(img, 100, 200)
    # plt.imshow(edges)
    # plt.show()
    # color the edges red in the img
    img[edges>0] = (255,0,0)
    # plt.imshow(img)
    # plt.show()

    return img





# # apply the preProcess function to all the images in the cnv_images list, dme_images list, drusen_images list and normal_images list
# cnv_images = [preProcess(img) for img in cnv_images]
# dme_images = [preProcess(img) for img in dme_images]
# drusen_images = [preProcess(img) for img in drusen_images]
# normal_images = [preProcess(img) for img in normal_images]





In [173]:
# create a labels list with 0 for cnv, 1 for dme, 2 for drusen and 3 for normal 
labelsMap = {"CNV":0, "DME":1, "DRUSEN":2, "NORMAL":3}

# create a dataframes list with all the images and labels in another column
import pandas as pd
df = pd.DataFrame()
df["images"] = cnv_images + dme_images + drusen_images + normal_images
df["labels"] = [labelsMap["CNV"]]*len(cnv_images) + [labelsMap["DME"]]*len(dme_images) + [labelsMap["DRUSEN"]]*len(drusen_images) + [labelsMap["NORMAL"]]*len(normal_images)

# shuffle the dataframe
df = df.sample(frac=1).reset_index(drop=True)




In [174]:
# create a cnn model for classification
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.layers import BatchNormalization
import numpy as np

# split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(df["images"], df["labels"], test_size=0.2, random_state=42)

# convert the images into numpy array
X_train = np.array(X_train.tolist())
X_test = np.array(X_test.tolist())

# reshape the data to 4 dimensions
X_train = X_train.reshape(X_train.shape[0], 256, 256, 3)
X_test = X_test.reshape(X_test.shape[0], 256, 256, 3)

# convert the labels into categorical format
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# create a sequential model
model = Sequential()
# add model layers
model.add(Conv2D(64, kernel_size=3, activation="relu", input_shape=(256,256,3)))
model.add(Conv2D(32, kernel_size=3, activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(16, kernel_size=3, activation="relu"))
model.add(Conv2D(8, kernel_size=3, activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(4, activation="softmax"))

# compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5)

# save the model to disk as eyemodel.h5
model.save("eyemodel.h5")

# create a model 2 with learning rate 0.0001
model2 = Sequential()
# add model layers
model2.add(Conv2D(64, kernel_size=3, activation="relu", input_shape=(256,256,3)))
# set learning rate to 0.001
lr = 0.0001
model2.add(Conv2D(32, kernel_size=3, activation="relu"))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())
model2.add(Dropout(0.25))
model2.add(Conv2D(16, kernel_size=3, activation="relu"))
model2.add(Conv2D(8, kernel_size=3, activation="relu"))

model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())
model2.add(Dropout(0.25))
model2.add(Flatten())
model2.add(Dense(4, activation="softmax"))

# compile the model
model2.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# train the model
model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5)

#






Epoch 1/5
100/100 [==============================] - 583s 6s/step - loss: 2.2113 - accuracy: 0.4328 - val_loss: 2.3697 - val_accuracy: 0.3125
Epoch 2/5
100/100 [==============================] - 594s 6s/step - loss: 0.9784 - accuracy: 0.7116 - val_loss: 2.1237 - val_accuracy: 0.4938
Epoch 3/5
100/100 [==============================] - 573s 6s/step - loss: 0.4616 - accuracy: 0.8462 - val_loss: 1.6200 - val_accuracy: 0.6325
Epoch 4/5
100/100 [==============================] - 547s 5s/step - loss: 0.2027 - accuracy: 0.9262 - val_loss: 1.7371 - val_accuracy: 0.5938
Epoch 5/5
1/1 [==============================] - 0s 281ms/step
[2 0 0 2 0]


25/25 [==============================] - 26s 1s/step
0.5925


In [6]:
# load the model
from keras.models import load_model
import cv2
import numpy as np
testmodel = load_model("eyeModel2.h5")
# print accuracy of the model
# print("Accuracy of the model is - " , testmodel.evaluate(X_test, y_test)[1]*100 , "%")

def predictImage(img):
    

    # preprocess the image
    img = preProcess(img)

    # convert the image into numpy array
    img = np.array(img.tolist())

    # reshape the image
    img = img.reshape(1, 256, 256, 3)

    # predict the image
    prediction = testmodel.predict(img)
    prediction = np.argmax(prediction, axis=1)
    # return prediction from labelsMap
    return list(labelsMap.keys())[list(labelsMap.values()).index(prediction[0])]

    # print the prediction
    print(np.argmax(prediction, axis=1))

img = cv2.imread("CNV-53018-1.jpeg")
print(predictImage(img))

# save model as .sav file
import pickle
pickle.dump(testmodel, open("eyeModel.sav", "wb"))


1/1 [==============================] - 0s 202ms/step


NameError: name 'labelsMap' is not defined

In [13]:
testmodel = load_model("eyeModel2.h5")

# user joblib to save model as .pkl file
import joblib
joblib.dump(testmodel, "eyeModel.pkl")

INFO:tensorflow:Assets written to: ram://e6581c89-cb13-4066-93b2-4a96fdd4a402/assets


INFO:tensorflow:Assets written to: ram://e6581c89-cb13-4066-93b2-4a96fdd4a402/assets


['eyeModel.pkl']

In [14]:

# load the model .pkl file
import joblib
testmodel = joblib.load("eyeModel.pkl")

# img = cv2.imread("CNV-53018-1.jpeg")
# img = preProcess(img)

#     # convert the image into numpy array
# img = np.array(img.tolist())

#     # reshape the image
# img = img.reshape(1, 256, 256, 3)

#     # predict the image
# prediction = testmodel.predict(img)
# prediction = np.argmax(prediction, axis=1)


FileNotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for ram://429a46b2-7de0-4b44-b9f6-fec6dbffb868/variables/variables
 You may be trying to load on a different device from the computational device. Consider setting the `experimental_io_device` option in `tf.saved_model.LoadOptions` to the io_device such as '/job:localhost'.